# Configuracion del directorio

In [1]:
import os

data_dirname = os.path.join(os.getcwd(), "data")
QUERY = "Reforma al poder judicial"

# Recoleccion de datos por paginas web
Se utiliza BeautifulSoup para el scraping, asi como 

In [2]:
import requests
from bs4 import BeautifulSoup
from googlesearch import search


def search_pages_google(query: str):
    return list(search(query, num_results=30))


def extract_page_content(url: str):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")

        for tag in soup(["script", "style", "noscript"]):
            tag.decompose()

        main_content = (
            soup.find("main")
            or soup.find("article")
            or soup.find("div", class_="content")
            or soup.find("div", class_="main")
        )
        if main_content:
            return main_content.get_text(strip=True, separator="\n")
    except Exception as e:
        print(f"Error al procesar {url}: {e}")
        return None


def search_in_pages():
    query = "Reforma Judicial Mexico"
    urls = search_pages_google(query)
    results = []
    for url in urls:
        content = extract_page_content(url)
        if content:
            results.append(content)


search_in_pages()

Error al procesar https://www.dof.gob.mx/nota_detalle.php?codigo=5738985&fecha=15/09/2024: HTTPSConnectionPool(host='www.dof.gob.mx', port=443): Max retries exceeded with url: /nota_detalle.php?codigo=5738985&fecha=15/09/2024 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))
Error al procesar https://www.cjf.gob.mx/reformaPJF/: HTTPSConnectionPool(host='www.cjf.gob.mx', port=443): Max retries exceeded with url: /reformaPJF/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))
Error al procesar https://law.stanford.edu/2024/11/05/la-reforma-judicial-en-mexico-viola-obligaciones-internacionales-en-materia-de-derechos-humanos/: 403 Client Error: Forbidden for url: https://law.stanford.edu/2024/11/05/la-reforma-judicial-en-mexico-viola-obligaciones-internacionales-

# Recolección de datos por pdf
Usando googlesearch en conjunto con requests, se extraen pdfs que sean despues del inicio de este anio, para verificar que efectivamente sea esta reforma.

In [3]:
import os
import requests
from googlesearch import search

pdf_dirname = os.path.join(data_dirname, "pdfs")


def download_pdf(url: str) -> None:
    try:
        response = requests.get(url)
        response.raise_for_status()
        filename = url.split("/")[-1]
        if not filename.endswith(".pdf"):
            filename += ".pdf"
        os.makedirs(pdf_dirname, exist_ok=True)
        filepath = os.path.join(pdf_dirname, filename)
        with open(filepath, "wb") as f:
            f.write(response.content)
    except Exception as e:
        print(f"Error al descargar {url}: {e}")


def search_pdfs():
    query = "Reforma Judicial Mexico filetype:pdf after:2024-01-01"
    results = search(query, num_results=30)
    for url in results:
        if url.endswith(".pdf"):
            download_pdf(url)


search_pdfs()

Error al descargar https://www.diputados.gob.mx/sedia/sia/spi/SAPI-ASS-11-24.pdf: HTTPSConnectionPool(host='www.diputados.gob.mx', port=443): Max retries exceeded with url: /sedia/sia/spi/SAPI-ASS-11-24.pdf (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))
Error al descargar https://escuelajudicial.cjf.gob.mx/MicroSitio/Doctos/Minuta_Reforma_Poder_Judicial.pdf: HTTPSConnectionPool(host='escuelajudicial.cjf.gob.mx', port=443): Max retries exceeded with url: /MicroSitio/Doctos/Minuta_Reforma_Poder_Judicial.pdf (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))
Error al descargar https://www.cjf.gob.mx/reformapjf/documentos/Jornadas_Nacionales_Reforma_Poder_Judicial.pdf: HTTPSConnectionPool(host='www.cjf.gob.mx', port=443): Max retries exceeded with url: /reform

## Transformacion de datos de pdf a txt

In [4]:
from PyPDF2 import PdfReader


def extract_text_from_pdf(filepath: str) -> str:
    try:
        text = ""
        with open(filepath, "rb") as f:
            reader = PdfReader(f)
            for page in reader.pages:
                text += page.extract_text()
                text += "\n"
        return text
    except Exception as e:
        return ""


def clean_text(text: str) -> str:
    lines = text.splitlines()
    cleaned_lines = [line.strip() for line in lines if line.strip()]
    return "\n".join(cleaned_lines)


def create_pdfs_txt():
    text = ""

    for filename in os.listdir(pdf_dirname):
        filepath = os.path.join(pdf_dirname, filename)
        text += extract_text_from_pdf(filepath)
    text = clean_text(text)
    output_file = os.path.join(data_dirname, "pdfs.txt")
    with open(output_file, "w", encoding="utf-8") as txt_file:
        txt_file.write(text)


create_pdfs_txt()

In [16]:
import whisper
model = whisper.load_model("base")

c:\Users\David\ia\.venv\Lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


# Recoleccion de datos por videos
Por medio de pytube.fix se descargan los videos, se extrae el audio del video con ffmpeg y con openai-whisper se pasa a un archivo de txt.


In [26]:
from pytubefix import YouTube
from pytubefix.cli import on_progress
from pytubefix.contrib.search import Search, Filter

audios_dirname = os.path.join(data_dirname, "audios")


def download_videos():
    global audios_dirname
    results = Search(
        query="Reforma Judicial Mexico",
        filters={"sort_by": Filter.get_sort_by("Relevance")},
    )
    i = 0
    for result in results.videos:
        try:
            yt = YouTube(result.watch_url, on_progress_callback=on_progress)
            video = yt.streams.get_audio_only()
            video.download(output_path=audios_dirname, filename=f"{i}.m4a")
            i += 1
        except:
            print(f"Error al descargar el video")


def transcript_audio(audio_path: str) -> str:
    try:

        result = model.transcribe(
            audio_path,
        )
        return result["text"]
    except Exception as e:
        print(f"Error al transcribir el audio: {e}")
        return None


def search_in_videos():
    global audios_dirname
    # download_videos()
    for filename in os.listdir(audios_dirname):
        filepath = os.path.join(audios_dirname, filename)
        text = transcript_audio(filepath)
        if text:
            print(text)


search_in_videos()

Error al transcribir el audio: [WinError 2] El sistema no puede encontrar el archivo especificado
Error al transcribir el audio: [WinError 2] El sistema no puede encontrar el archivo especificado
Error al transcribir el audio: [WinError 2] El sistema no puede encontrar el archivo especificado
Error al transcribir el audio: [WinError 2] El sistema no puede encontrar el archivo especificado
Error al transcribir el audio: [WinError 2] El sistema no puede encontrar el archivo especificado
Error al transcribir el audio: [WinError 2] El sistema no puede encontrar el archivo especificado
Error al transcribir el audio: [WinError 2] El sistema no puede encontrar el archivo especificado
Error al transcribir el audio: [WinError 2] El sistema no puede encontrar el archivo especificado
Error al transcribir el audio: [WinError 2] El sistema no puede encontrar el archivo especificado
Error al transcribir el audio: [WinError 2] El sistema no puede encontrar el archivo especificado
Error al transcribir